In [1]:
# Import required modules
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from keras import backend as K
from keras.layers import LSTM
from keras.models import Sequential, Model
from keras.layers import *
from keras.optimizers import RMSprop
from keras.regularizers import l2
from generator import Generator
from models import *
import h5py
import numpy as np
import sys
import os

# Set Tensorflow backend to avoid full GPU pre-loading
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config = config))

Using TensorFlow backend.


In [2]:
print "Loading Generator"
# Hardcoding because codes will mostly run from my account.
# Also adding sys.argv stuff with defaults is a pain.
gen = Generator(dataset_directory = '/home/siddhartha.l/Show-and-Tell/data')

Loading Generator


In [3]:
print "Generating model (This will take while)"
model = get_model_with_attention(
            gen.img_feature_size, 
            gen.vocab_size, 
            gen.max_token_len, 
            embedding_dim = gen.embedding_size)
model.compile('adamax', loss='categorical_crossentropy', metrics=['accuracy'])

Generating model (This will take while)


In [4]:
batch_size = 64
print "Start training"
# Save model for every 10 epochs
prev_loss = 10
for i in range(1,10):
    hist = model.fit_generator(
                gen.pullData(),
                epochs=10,
                steps_per_epoch=int(gen.training_samples_count / batch_size), 
                shuffle = True, verbose = 1#, callbacks = [checkpointer]
                )
    if hist.history['loss'][-1] < prev_loss:
        prev_loss = hist.history['loss'][-1]
        print 'Saving ~models/attention_flickr_epoch_' + str(i*10) + '.h5'
        model.save('/home/siddhartha.l/Show-and-Tell/data/models/attention_flickr_epoch_' + str(i*10) + '.h5')

Start training
Epoch 1/10
632/632 [==============================] - 316s - loss: 1.1617 - acc: 0.7968   
Epoch 2/10
632/632 [==============================] - 315s - loss: 0.8737 - acc: 0.8215   
Epoch 3/10
632/632 [==============================] - 315s - loss: 0.7721 - acc: 0.8322   
Epoch 4/10
632/632 [==============================] - 311s - loss: 0.6852 - acc: 0.8427   
Epoch 5/10
632/632 [==============================] - 310s - loss: 0.6241 - acc: 0.8508   
Epoch 6/10
632/632 [==============================] - 309s - loss: 0.5635 - acc: 0.8603   
Epoch 7/10
632/632 [==============================] - 314s - loss: 0.5073 - acc: 0.8696   
Epoch 8/10
632/632 [==============================] - 312s - loss: 0.4700 - acc: 0.8764   
Epoch 9/10
632/632 [==============================] - 315s - loss: 0.4334 - acc: 0.8838   
Epoch 10/10
632/632 [==============================] - 313s - loss: 0.3958 - acc: 0.8917   
Saving ~models/attention_flickr_epoch_10.h5
Epoch 1/10
632/632 [==========

RuntimeError: Can't decrement id ref count (unable to close file, errno = 122, error message = 'Disk quota exceeded')

In [ ]:
model.save('../data/att_tmp.h5')

In [ ]:
# Load previously saved model if necessary
from keras.models import load_model

model = load_model('../data/models/attention_flickr_epoch_20.h5')

In [ ]:
from IPython.display import Image
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input

In [ ]:
### For runtime testing only ###

# from process_images import *

base_model = InceptionV3(weights = 'imagenet', include_top = True, input_shape = (299, 299, 3))
img_model =  Model(
        input = base_model.input,
        outputs = [base_model.get_layer('mixed10').output])
target_size = (299, 299)
print img_model.output_shape
output_shape = img_model.output_shape[1:]

In [ ]:
model.summary()

In [ ]:
image_filenames = ['2021613437_d99731f986.jpg']
dataset_directory = '../data/flicker8k'

from keras.backend.tensorflow_backend import get_session

preprocessed_images = []
# Remove repeating filenames (if any)
# image_filenames = list(set(image_filenames))
number_of_images = len(image_filenames)
img_input = []

# Iterate over all images and preprocess them
for img_id, img_name in enumerate(image_filenames): # For coco make 3D array , do batch
    img_filepath = dataset_directory + '/Flickr8k_Dataset/' + img_name
    # Image preprocessing
    img = image.load_img(img_filepath, target_size = target_size)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)

    preprocessed_images.append(np.squeeze(img))

preprocessed_images = np.asarray(preprocessed_images)
img_features = img_model.predict(preprocessed_images)
# img_features = np.asarray([img_features])

text_in = np.zeros((1,gen.max_token_len))
text_in[0][0] = gen.token_to_id['<start>']

predictions = []
activations = []
for arg in range(gen.max_token_len-1):
    pred = model.predict([img_features, text_in])
    tok = np.argmax(pred[0][arg])
    word = gen.id_to_token[tok]
#     if word == 'boy':
#         att_map = model.get_layer('time_distributed_6').output
#         att_map = K.reshape(att_map, (39, 8, 8))
#         att_map = att_map.eval(session = get_session())
    text_in[0][arg+1] = tok
    if word == '<end>':
        break
    predictions.append(word)

print ' '.join(predictions) + '.'
# print att_map
Image(filename= dataset_directory + '/Flickr8k_Dataset/' + image_filenames[0])

In [ ]:
image_filenames = ['2021613437_d99731f986.jpg']
dataset_directory = '../data/flicker8k'

from matplotlib.pyplot import imshow
from matplotlib.pyplot import imread
from matplotlib import pyplot
% matplotlib inline

preprocessed_images = []
# Remove repeating filenames (if any)
# image_filenames = list(set(image_filenames))
number_of_images = len(image_filenames)
img_input = []

# Iterate over all images and preprocess them
for img_id, img_name in enumerate(image_filenames): # For coco make 3D array , do batch
    img_filepath = dataset_directory + '/Flickr8k_Dataset/' + img_name
    # Image preprocessing
    img = image.load_img(img_filepath, target_size = target_size)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)

    preprocessed_images.append(np.squeeze(img))

preprocessed_images = np.asarray(preprocessed_images)
img_features = img_model.predict(preprocessed_images)
# img_features = np.asarray([img_features])

text_in = np.zeros((1,gen.max_token_len))
text_in[0][0] = gen.token_to_id['<start>']
text_in[0][1] = gen.token_to_id['a']
text_in[0][2] = gen.token_to_id['little']
# text_in[0][3] = gen.token_to_id['and']
# text_in[0][4] = gen.token_to_id['a']

layer_name = 'time_distributed_6'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
# intermediate_output = intermediate_layer_model.predict(data)

predictions = []
activations = []
# for arg in range(gen.max_token_len-1):
pred = intermediate_layer_model.predict([img_features, text_in])
#     tok = np.argmax(pred[0][arg])
#     word = gen.id_to_token[tok]
#     text_in[0][arg+1] = tok
#     if word == '<end>':
#         break
#     predictions.append(word)

# print ' '.join(predictions) + '.'
# print att_map
# Image(filename= dataset_directory + '/Flickr8k_Dataset/' + image_filenames[0])
# print pred.shape
# pyplot.subplot()
att = pred[0,3,:].reshape((8,8), order = 'A')#, np.ones((immmg.shape[0], immmg.shape[1])))
# att.shape
immmg = imread(dataset_directory + '/Flickr8k_Dataset/' + image_filenames[0])
imshow(immmg)
imshow(att, alpha = 0.5, interpolation='bilinear', extent=[0, immmg.shape[1], immmg.shape[0], 0])
# imshow(pred[0,2,:].reshape(8,8))
# imshow(pred[0,1,:].reshape(8,8))

In [ ]:
imshow(pred[0,0,:].reshape(8,8))

In [ ]:
imshow(pred[0,0,:].reshape(8,8))

In [ ]:
# For generating test results in bulk

from IPython.display import Image
import time

with open('../data/flicker8k/preprocessed/test_captions.txt') as captions_file:
    captions = captions_file.read().split('\n')
    
class Caption:
    def __init__(self, name):
        self.name = name
        self.captions = ['','','','','']
        self.result = ''
        
    def add(self, caption_number, caption):
        self.captions[caption_number] = caption
        
test_results = {}

for caption in captions:
    if len(caption) < 5:
        continue
    caption = caption.split('\t')
    img_name = caption[0].split('#')
    caption_number = int(img_name[1])
    img_name = img_name[0]
    caption = caption[1].lower()
    
    try:
        cap_obj = test_results[img_name]
        cap_obj.add(caption_number, caption)
    except Exception as e:
#         print str(e)
        feature_dataset = h5py.File('../data/flicker8k/preprocessed/test_features.h5', 'r')
        img_features = feature_dataset[img_name]['cnn_features'][:]

        # image_filenames = get_image_filenames(dataset_directory + '/' + img_list_file)

        # print img_features.shape
        features = np.array([img_features])

        text_in = np.zeros((1,gen.max_token_len))
        text_in[0][:] = np.full((gen.max_token_len,), 0)
        text_in[0][0] = 4230

        # print features,text_in
        arr = []
        zeros = np.array([np.zeros(512)])
        for arg in range(gen.max_token_len-1):
            pred = model.predict([features, zeros, text_in])
            tok = np.argmax(pred[0][arg])
            word = gen.id_to_token[tok]
            text_in[0][arg+1] = tok
            if word == '<end>':
                break
            arr.append(word)

        arr.append('.')
        cap_obj = Caption(img_name)
        cap_obj.add(caption_number, caption)
        cap_obj.result = ' '.join(arr)
        test_results.update({img_name: cap_obj})
        
        import pickle
        pickle.dump(test_results, open('../data/flicker8k/preprocessed/test_results.p', 'wb') )

In [ ]:
# Sample code for importing tested results
import pickle

class Caption:
    def __init__(self, name):
        self.name = name
        self.captions = ['','','','','']
        self.result = ''
        
    def add(self, caption_number, caption):
        self.captions[caption_number] = caption
        
test_results = pickle.load(open('../data/flicker8k/preprocessed/test_results.p', 'rb'))

for img_name in test_results:
    ground_truth = test_results[img_name].captions
    result = test_results[img_name].result